# <p style="background-color:black; font-family:'Pacifico', cursive; color:yellow; font-size:140%; text-align:center; border: 2px solid black; border-radius:15px; padding: 10px; box-shadow: 5px 5px 20px rgba(0, 0, 0, 0.5); font-weight: bold; letter-spacing: 1px;">XGB || Forecasting Sticker Sales</p>

In [3]:
%%time

!pip install -qq lifelines
!pip install -qq hillclimbers

import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from colorama import Fore
from IPython.display import clear_output

from sklearn.model_selection import *
from xgboost import XGBRegressor, XGBClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
import catboost as cb
from lightgbm import LGBMRegressor
import lightgbm as lgb
from tqdm import tqdm
import numpy as np

CPU times: user 220 ms, sys: 70.9 ms, total: 291 ms
Wall time: 9.48 s


# <p style="background-color:black; font-family:'Pacifico', cursive; color:yellow; font-size:140%; text-align:center; border: 2px solid black; border-radius:15px; padding: 10px; box-shadow: 5px 5px 20px rgba(0, 0, 0, 0.5); font-weight: bold; letter-spacing: 1px;">Load Data</p>

In [4]:
%%time

SEED = 42
n_splits = 5

!git clone https://github.com/muhammadabdullah0303/AbdML

import sys
sys.path.append('/kaggle/working/repository')

from AbdML.main import AbdBase

train = pd.read_csv('/kaggle/input/playground-series-s5e1/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s5e1/test.csv')
sample = pd.read_csv('/kaggle/input/playground-series-s5e1/sample_submission.csv')
train = train.dropna(subset=['num_sold'])

train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

cat_c = ['country', 'store', 'product','month_name','day_of_week']

ohe_cols = {'cat_c': cat_c}


base = AbdBase(train_data=train, test_data=test, target_column='num_sold',gpu=False,handle_date=True,
                 problem_type="regression", metric="mape", seed=SEED,
                 n_splits=n_splits,early_stop=False,num_classes=0,cat_features=None,ordinal_encoder=ohe_cols,
                 fold_type='GKF')

base.X_train = base.X_train.fillna(-1)
base.X_test = base.X_test.fillna(-1)


Cloning into 'AbdML'...
remote: Enumerating objects: 292, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 292 (delta 46), reused 0 (delta 0), pack-reused 147 (from 1)
Receiving objects: 100% (292/292), 109.14 KiB | 5.20 MiB/s, done.
Resolving deltas: 100% (93/93), done.
*** AbdBase ['V_1.3'] ***

 *** Available Settings *** 

Available Models: LGBM, CAT, XGB, Voting, TABNET, Ridge, LR
Available Metrics: roc_auc, accuracy, f1, precision, recall, rmse, wmae, rmsle, mae, r2, mse, mape, custom
Available Problem Types: classification, regression
Available Fold Types: SKF, KF, GKF, GSKF, RKF

 *** Configuration *** 

Problem Type Selected: REGRESSION
Metric Selected: MAPE
Fold Type Selected: GKF
Calculate Train Probabilities: False
Calculate Test Probabilities: False
Early Stopping: False
GPU: False
Eval_Metric Selected is: None

Adding Date Features

---> Applying Ordinal Encoder

CPU times: user 2.01 s, sys: 429 ms, tot

In [5]:
%%time

base.X_train.head()

CPU times: user 211 µs, sys: 0 ns, total: 211 µs
Wall time: 216 µs


,country,store,product,year,day,month,month_name,day_of_week,week,month_sin,month_cos,day_sin,day_cos,group,cos_year,sin_year,year_lag_1,year_diff
1,0.0,0.0,1.0,2010,1,1,4.0,0.0,53,0.5,0.866025,0.201299,0.97953,-476,0.809017,0.587785,-1.0,-1.0
2,0.0,0.0,2.0,2010,1,1,4.0,0.0,53,0.5,0.866025,0.201299,0.97953,-476,0.809017,0.587785,2010.0,0.0
3,0.0,0.0,3.0,2010,1,1,4.0,0.0,53,0.5,0.866025,0.201299,0.97953,-476,0.809017,0.587785,2010.0,0.0
4,0.0,0.0,4.0,2010,1,1,4.0,0.0,53,0.5,0.866025,0.201299,0.97953,-476,0.809017,0.587785,2010.0,0.0
5,0.0,2.0,0.0,2010,1,1,4.0,0.0,53,0.5,0.866025,0.201299,0.97953,-476,0.809017,0.587785,2010.0,0.0


In [6]:
%%time

base.X_test.head()

CPU times: user 162 µs, sys: 27 µs, total: 189 µs
Wall time: 193 µs


,country,store,product,year,day,month,month_name,day_of_week,week,month_sin,month_cos,day_sin,day_cos,group,cos_year,sin_year,year_lag_1,year_diff
0,0.0,0.0,0.0,2017,1,1,4.0,3.0,52,0.5,0.866025,0.201299,0.97953,-140,0.481754,0.876307,-1.0,-1.0
1,0.0,0.0,1.0,2017,1,1,4.0,3.0,52,0.5,0.866025,0.201299,0.97953,-140,0.481754,0.876307,2017.0,0.0
2,0.0,0.0,2.0,2017,1,1,4.0,3.0,52,0.5,0.866025,0.201299,0.97953,-140,0.481754,0.876307,2017.0,0.0
3,0.0,0.0,3.0,2017,1,1,4.0,3.0,52,0.5,0.866025,0.201299,0.97953,-140,0.481754,0.876307,2017.0,0.0
4,0.0,0.0,4.0,2017,1,1,4.0,3.0,52,0.5,0.866025,0.201299,0.97953,-140,0.481754,0.876307,2017.0,0.0


# <p style="background-color:black; font-family:'Pacifico', cursive; color:yellow; font-size:140%; text-align:center; border: 2px solid black; border-radius:15px; padding: 10px; box-shadow: 5px 5px 20px rgba(0, 0, 0, 0.5); font-weight: bold; letter-spacing: 1px;">Base XGB</p>

In [7]:
%%time
params =  {'n_estimators': 2203, 'max_depth': 5, 'colsample_bytree': 0.5359752614980476,
            'subsample': 0.7271274739921461, 'learning_rate': 0.011247656752870117, 'min_child_weight': 74}

XGBresult = base.Train_ML(params,'XGB', y_log=True, g_col='year')

Training Folds: 100%|██████████| 5/5 [03:15<00:00, 39.11s/it]

Overall Train MAPE: 0.0457
Overall OOF MAPE: 0.0859 
CPU times: user 12min 37s, sys: 2.08 s, total: 12min 39s
Wall time: 3min 15s


In [8]:
%%time
params =  {'n_estimators': 2203, 'max_depth': 5, 'colsample_bytree': 0.5359752614980476,
            'subsample': 0.7271274739921461, 'learning_rate': 0.011247656752870117, 'min_child_weight': 74}

XGBresult = base.Train_ML(params,'XGB', y_log=False, g_col='year')

Training Folds: 100%|██████████| 5/5 [03:14<00:00, 38.94s/it]

Overall Train MAPE: 0.5495
Overall OOF MAPE: 0.5801 
CPU times: user 12min 28s, sys: 1.95 s, total: 12min 30s
Wall time: 3min 14s


# <p style="background-color:black; font-family:'Pacifico', cursive; color:yellow; font-size:140%; text-align:center; border: 2px solid black; border-radius:15px; padding: 10px; box-shadow: 5px 5px 20px rgba(0, 0, 0, 0.5); font-weight: bold; letter-spacing: 1px;">Submission</p>

In [9]:
%%time

sample["num_sold"] = XGBresult[1]
sample.to_csv("submission.csv", index=False)
print("Sub shape:", sample.shape)
sample.head()

Sub shape: (98550, 2)
CPU times: user 199 ms, sys: 8.99 ms, total: 208 ms
Wall time: 208 ms


,id,num_sold
0,230130,91.450479
1,230131,951.577148
2,230132,787.048340
3,230133,385.042474
4,230134,474.187579
